# Protocole 2 : Vérification des Performances d’une Photodiode Activée par Arduino

Ce protocole a pour but de vérifier la précision et la stabilité temporelle d’une photodiode lorsqu’elle est éclairée par une LED contrôlée par une carte Arduino.


### Prérequis

**Matériel :**
* 1 × Arduino (Nano, Uno ou équivalent)  
* 1 × LED blanche  
* 1 × Résistance 220 Ω (en série avec la LED)  
* 1 × Photodiode + son circuit d’alimentation/lecture  
* 1 × Breadboard et câbles Dupont  
* 1 × Ordinateur avec **Arduino IDE** pour flasher le code  
* 1 × Ordinateur avec **Python 3** et les bibliothèques nécessaires (`numpy`, `matplotlib`, `scipy`)  

**Codes nécessaires :**
* `arduino_trigger_master.ino` (sketch Arduino utilisé ici pour basculer le pin HIGH/LOW et contrôler la LED)  
* `arduino_led_master.py` (optionnel, si pilotage série)  
* `analyser_et_tracer_photodiode_wav.py` (script Python pour analyser les mesures de la photodiode)  


### Étapes à suivre

#### 1. Préparation du Matériel

1. Branchez une **LED** et une **résistance de 220 Ω** en série entre le **pin 8** de l’Arduino et le **GND**.  
2. Placez la **photodiode** face à la LED, à une distance d’environ **5 cm**.  
3. Vérifiez l’alimentation et la lecture correcte de la photodiode.  
4. Préparez deux environnements de test :  
   * **Quasi-obscurité** (pièce sombre)  
   * **Conditions réelles** (lumière ambiante allumée)  


#### 2. Programmation de l’Arduino

1. Flashez sur l’Arduino un code qui fait basculer le **pin de sortie** (auquel la LED est branchée) de l’état **HIGH** à **LOW** de manière répétée.  
2. Dans notre cas, nous utilisons le sketch `arduino_trigger_master.ino`, initialement conçu pour générer des triggers dans d’autres expériences.  
3. Ce code n’est pas optimisé spécifiquement pour cette manipulation car il contient d’autres fonctions, mais il reste adapté pour allumer et éteindre la LED avec précision.  

    > **Code Arduino (.ino) pour la génération périodique automatique de triggers via TimerOne.**
    >
    >     // Utilise la librairie TimerOne : https://www.pjrc.com/teensy/td_libs_TimerOne.html
    >     #include <TimerOne.h>
    >
    >     const int triggerPin = 8;       // Sortie TTL sur la pin 8
    >     const unsigned long period_us = 500000;   // Période = 500 ms
    >     const unsigned long pulse_width_us = 1000; // Largeur du pulse = 1 ms
    >
    >     void setup() {
    >       pinMode(triggerPin, OUTPUT);
    >       digitalWrite(triggerPin, LOW);
    >
    >       Timer1.initialize(period_us);         // Timer déclenché toutes les 500 ms
    >       Timer1.attachInterrupt(trigger);      // Fonction appelée à chaque tick
    >     }
    >
    >     void loop() {
    >       // Le timer fait tout, rien à faire ici
    >     }
    >
    >     void trigger() {
    >       digitalWrite(triggerPin, HIGH);
    >       delayMicroseconds(pulse_width_us);    // Pulse de 1 ms
    >       digitalWrite(triggerPin, LOW);
    >     }


#### 3. Acquisition des Signaux

1. **Condition 1 : pièce sombre, bras immobile**  
   Lancez la séquence Arduino et enregistrez la sortie de la photodiode.  
2. **Condition 2 : pièce sombre, bras en mouvement léger**  
   Reproduisez l’enregistrement et observez l’impact du mouvement.  
3. **Condition 3 : lumière ambiante, bras immobile**  
   Reproduisez l’enregistrement en pleine lumière.  


Sauvegardez les fichiers avec une nomenclature claire, par exemple:  
* `dark_immobile.wav`  
* `dark_move.wav`  
* `light_immobile.wav` 

#### 4. Analyse des Signaux

1. Utilisez le script Python `analyser_et_tracer_photodiode_wav.py` afin de :  
   * Détecter les pics lumineux  
   * Calculer les intervalles ON/OFF  
   * Générer des histogrammes et des tracés du signal  

    > **Code Python (.py) pour l’analyse du signal de la photodiode.**
> ```python    
> import numpy as np
> import matplotlib
> matplotlib.use('Agg')  # Backend non interactif
> import matplotlib.pyplot as plt
> from scipy.io import wavfile
> from scipy.signal import find_peaks
> import os
> 
> # --- Paramètres ---
> NOM_FICHIER_WAV = 'psychopy_stimuli_only_250ms.wav'  # Nom du fichier WAV à analyser
> DISTANCE_MIN_MS = 100.0                       # Intervalle minimal entre pics
> 
> def analyser_et_sauvegarder_graphique(nom_fichier):
>     """
>     Analyse un signal photodiode au format WAV :
>     - détection des pics
>     - calcul des intervalles
>     - histogramme des intervalles
>     - tracé du signal avec pics
>     """
>     try:
>         sampling_rate, data = wavfile.read(nom_fichier)
>         print(f"\n--- Analyse du fichier : {nom_fichier} ---")
>         print(f"Taux d'échantillonnage : {sampling_rate} Hz")
> 
>         if data.ndim > 1:
>             data = data[:, 0]
> 
>         signal_abs = np.abs(data)
>         seuil = 0.8 * np.max(signal_abs)
>         print(f"Seuil de détection (80% amplitude max) : {int(seuil)}")
> 
>         distance_min_samples = int(DISTANCE_MIN_MS * sampling_rate / 1000)
> 
>         indices_pics, _ = find_peaks(signal_abs, height=seuil, distance=distance_min_samples)
>         intervalles_en_samples = np.diff(indices_pics)
>         intervalles_en_ms = (intervalles_en_samples / sampling_rate) * 1000
> 
>         print(f"\nNombre de pics détectés : {len(indices_pics)}")
>         print(f"Nombre d'intervalles : {len(intervalles_en_ms)}")
> 
>         if len(intervalles_en_ms) > 0:
>             moyenne = np.mean(intervalles_en_ms)
>             print(f"Intervalle moyen : {moyenne:.2f} ms")
>             print(f"Min : {np.min(intervalles_en_ms):.2f} ms")
>             print(f"Max : {np.max(intervalles_en_ms):.2f} ms")
> 
>             # --- Histogramme des intervalles ---
>             plt.figure(figsize=(12, 6))
>             plt.hist(intervalles_en_ms, bins=50, edgecolor='black', alpha=0.75)
>             plt.axvline(moyenne, color='red', linestyle='--', linewidth=2, label=f'Moyenne = {moyenne:.2f} ms')
>             plt.title(f"Distribution des intervalles — {os.path.basename(nom_fichier)}", fontsize=14)
>             plt.xlabel("Durée de l'intervalle (ms)")
>             plt.ylabel("Nombre d'intervalles")
>             plt.legend()
>             plt.grid(True, linestyle='--', alpha=0.6)
> 
>             output_dir = 'figures'
>             os.makedirs(output_dir, exist_ok=True)
>             base_name = os.path.splitext(os.path.basename(nom_fichier))[0]
>             save_hist_path = os.path.join(output_dir, f'distribution_intervalles_{base_name}.png')
>             plt.savefig(save_hist_path, dpi=300, bbox_inches='tight')
>             plt.close()
>             print(f"\nHistogramme sauvegardé : {save_hist_path}")
> 
>             # --- Tracé du signal avec pics ---
>             plt.figure(figsize=(15, 5))
>             plt.plot(data, label='Signal')
>             plt.plot(indices_pics, data[indices_pics], 'rx', label='Pics détectés')
>             plt.title(f"Signal et pics détectés — {base_name}")
>             plt.xlabel("Échantillons")
>             plt.ylabel("Amplitude")
>             plt.legend()
>             save_signal_path = os.path.join(output_dir, f'signal_pics_{base_name}.png')
>             plt.savefig(save_signal_path, dpi=300, bbox_inches='tight')
>             plt.close()
>             print(f"Tracé du signal sauvegardé : {save_signal_path}")
> 
>         else:
>             print("Aucun intervalle détecté — pas de figure générée.")
> 
>     except FileNotFoundError:
>         print(f"ERREUR : Le fichier '{nom_fichier}' est introuvable.")
>     except Exception as e:
>         print(f"Une erreur est survenue : {e}")
> 
> # --- Lancement automatique ---
> if __name__ == '__main__':
>     analyser_et_sauvegarder_graphique(NOM_FICHIER_WAV)
> ```


#### 5. Comparaison des Conditions

Comparez les intervalles mesurés avec la valeur attendue (ex. **250 ms de période, 100 ms ON**).  
Évaluez :  
* **Intervalle moyen**  
* **Valeurs min / max**  
* **Écart-type (jitter)** 
Ici suivent les performances obtenues pour les mesures dans une salle éclairée puis dans le noir (avec et sans mouvement) :   
>![Scope settings](../pictures/arduino_trigger_master_timer_led_light_on.png)
>![Scope settings](../pictures/arduino_trigger_master_timer_led_move.png)
>![Scope settings](../pictures/arduino_trigger_master_timer_led_no_move.png)


#### 6. Interprétation

* Vérifiez si la lumière ambiante provoque du **bruit** et réduit la précision de détection.  
* Analysez l’influence des mouvements du bras sur la **stabilité temporelle**.  
* Notez que les résultats peuvent varier de quelques millisecondes selon les conditions expérimentales.  

Ici suivent les résultats obtenus pour les mesures dans une salle éclairée puis dans le noir :   
>![Scope settings](../pictures/results_plot_led_light.png)
>![Scope settings](../pictures/results_plot_led.png)
